In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import train_test_split
import re
from string import punctuation

In [317]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import re
from string import punctuation
from collections import Counter
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
#nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to C:\Users\l-Ju
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [318]:
path='C:/Users/l-Ju Chen/Documents/GitHub/Kaggle_tweet_sentiment/'
data=pd.read_csv(path+'train.csv')
test_data=pd.read_csv(path+'test.csv')
sample_submission=pd.read_csv(path+'sample_submission.csv')

In [319]:
data.dropna(axis = 0, how ='any',inplace=True)

In [371]:
print(set(data['sentiment']))
y=data['selected_text']
X=data.drop(columns=['selected_text'])
data.head(n=10)

{'neutral', 'negative', 'positive'}


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


In [321]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.15)

In [322]:
def clean(text):
    #text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    #text = re.sub('[%s]' % re.escape(punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text.lower()

In [359]:
def get_column_from_sentiment(sentiment):
    selected_data=X_train.loc[X_train['sentiment'] == sentiment]
    return selected_data['text']
def get_word_count(column):
    all_text="".join(column)
    all_text=clean(all_text)
    all_text=all_text.split(' ')
    all_text = list(filter(None, all_text))
    #all_text = [WordNetLemmatizer().lemmatize(w, pos='v') for w in all_text]
    #all_text = [w for w in all_text if w not in stopwords.words("english")]
    total=len(all_text)
    counts = Counter(all_text)
    for item, count in counts.items():
        counts[item] /= total
    return counts

In [360]:
text=X_train['text']
selected_text=y_train

text="".join(text)
selected_text="".join(selected_text)

text=clean(text)
selected_text=clean(selected_text)
#print(clean_all_positive_text)
text=text.split(' ')
selected_text=selected_text.split(' ')

text = list(filter(None, text))
selected_text = list(filter(None, selected_text))

text_counter = get_word_count(text)

list_difference = [item for item in text if item not in selected_text]

In [365]:
print('2' in selected_text)

False


In [356]:
positive_counter=get_word_count(get_column_from_sentiment('positive'))
#print(positive_counter)

In [357]:
negative_counter=get_word_count(get_column_from_sentiment('negative'))
#print(negative_counter)

In [358]:
neutral_counter=get_word_count(get_column_from_sentiment('neutral'))
#print(neutral_counter)

In [328]:
positive_list=list(positive_counter.keys())
negative_list=list(negative_counter.keys())
neutral_list=list(neutral_counter.keys())
common_elements = list(set(positive_list).intersection(set(negative_list)))
common_elements = list(set(common_elements).intersection(set(neutral_list)))
#print(common_elements)

In [350]:
no_words=[]
neighbor=0
for w in common_elements:
    if (positive_counter[w]> (1-neighbor)*negative_counter[w] and positive_counter[w]< (1+neighbor)*negative_counter[w]) and (neutral_counter[w]> (1-neighbor)*negative_counter[w] and neutral_counter[w]< (1+neighbor)*negative_counter[w]):
        no_words.append(w)
print(no_words)
    

[]


In [351]:
remove_list=set(list_difference+no_words)
#no_words+
#print(remove_list)

In [366]:
def predict(sentence):
    sentence=clean(sentence)
    words=sentence.lower().split()
    selected_words=[w for w in words if w not in remove_list]
    selected_words=' '.join(selected_words)
    return selected_words
    
select=predict('Sooo SAD I will miss you here in San Diego!!!')
#print(select)

In [367]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [368]:
jac=[]
for i in range(X_test.shape[0]): 
    sentence=X_test['text'].iloc[i]
    selected_sentence=y_test.iloc[i]
    if len(sentence.split())>2:
        prediction=predict(sentence)
    else:
        prediction
    jac.append(jaccard(selected_sentence,prediction))

print(np.mean(jac))

0.46113729399963677


In [306]:
print(sample_submission.head())

       textID                                      selected_text
0  f87dea47db                            last session of the day
1  96d74cb729  shanghai is also really exciting (precisely --...
2  eee518ae67  recession hit veronique branquinho, she has to...
3  01082688c6                                        happy bday!
4  33987a8ee5                                           - i like


In [298]:
sample_submission=sample_submission.drop('selected_text',axis=1)
for i in range(test_data.shape[0]):
    sentence=test_data['text'].iloc[i]
    if len(sentence.split())>2:
        prediction=predict(sentence)
    else:
        prediction=sentence
    sample_submission.at[i,'selected_text']=prediction

In [299]:
sample_submission.to_csv('sample_submission.csv',index=False)

In [300]:
sample_submission.head()

,textID,selected_text
0,f87dea47db,last session of the day
1,96d74cb729,shanghai is also really exciting (precisely --...
2,eee518ae67,"recession hit veronique branquinho, she has to..."
3,01082688c6,happy bday!
4,33987a8ee5,- i like
